In [2]:
!!apt install tesseract-ocr
!pip install pytesseract pillow

In [9]:
import pytesseract
from PIL import Image
import os

In [5]:
def extract_text(image_path):
    with Image.open(image_path) as img:
        zoom_factor = 2
        img = img.resize((img.width * zoom_factor, img.height * zoom_factor))
        text = pytesseract.image_to_string(img)
        return text

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope and credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/artful-sky-413615-b3887267dc62.json', scope)

# Authorize the client
client = gspread.authorize(credentials)

# Create a new Google Sheets file
spreadsheet = client.create('My Spreadsheet')

# Share the spreadsheet (optional)
spreadsheet.share('recipient@example.com', perm_type='user', role='writer')

# Get the URL of the created spreadsheet
spreadsheet_url = spreadsheet.url

print(f"Spreadsheet created successfully! URL: {spreadsheet_url}")


In [ ]:
import os
from transformers import pipeline
ner = pipeline("ner", grouped_entities=True)
import logging

logging.getLogger("transformers.configuration_utils").setLevel(logging.ERROR)
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

file_path = "/content/drive/MyDrive/Publications_text/Train/"
res = ""
for files in os.listdir(file_path):
  # Read text from file
    text_path = os.path.join(file_path, files)
    with open(text_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified_lines = []
    for i, line in enumerate(lines):
        if i < 10:
            modified_lines.append(line.replace('\n', '. '))
        else:
            modified_lines.append(line.replace('\n', ' '))

    # Join the modified lines back into a single string
    modified_text = ''.join(modified_lines)
    context=modified_text
    # print(context)
    entities = ner(context)
    persons = [entity["word"] for entity in entities if entity['entity_group'] == 'PER' and entity["start"]]
    if(len(persons)==0):
      persons = [entity["word"] for entity in entities if entity['entity_group'] == 'PER' and entity["start"]]
    authors = " ".join(persons)
    text_generator = pipeline("text-generation")
    qa = pipeline("question-answering")
    context = context[:200]
    answer = qa(
        question = "What is the title of the publication?",
        context = context,
        max_length = 8
    )["answer"]
    title = answer
    answer = qa(
        question = "Generate another alternative title",
        context = answer
    )
    new_title = answer["answer"]
    res += authors+'|'+ title+'|'+ new_title + '\n'

print(res)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [14]:
qa = pipeline("question-answering")
context = context[:200]
answer = qa(
    question = "What is the title of the publication?",
    context = context,
    max_length = 8
)["answer"]
print(answer)
answer = qa(
    question = "Generate another alternative title",
    context = answer
)
print(answer["answer"])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Journal of Chromatographic Science
Journal of Chromatographic Science


In [50]:
from transformers import pipeline

summarizer = pipeline("summarization")
ans = summarizer(answer)[0]
ans = ans['summary_text']
print(ans)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 142, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


 Solanesol: A Tracer tor Environmental Tobacco Smoke Particles law . The law requires the use of solanesol to detect smoke particles in smoke smoke particles . It is the first time a law has been passed in the U.S. Congress has passed a law that requires a tracer for smoke particles to be detected .


In [ ]:
from transformers import pipeline

# Initialize the text generation pipeline
text_generator = pipeline("text-generation")

paragraph = """
The sun rose slowly over the horizon, casting a golden hue over the tranquil meadow. Birds chirped cheerfully as they flitted from tree to tree, and a gentle breeze rustled the leaves. In the distance, the sound of a babbling brook added to the peaceful melody of nature.
"""

# Generate a title
generated_title = text_generator(paragraph, max_length=10, num_return_sequences=1)

print(generated_title[0]['generated_text'].strip())